#  Assignment_4-Ragul M,1P24CS014

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import train_test_split

1.A


In [17]:
data = load_data('Default')
data

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


In [18]:
x=data[['income','balance']]
y=data['default']=="Yes"
x_i = MS(x).fit_transform(x)
model=sm.GLM(y,x_i,family=sm.families.Binomial()).fit()
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -789.48
Date:                Wed, 15 Oct 2025   Deviance:                       1579.0
Time:                        14:06:53   Pearson chi2:                 6.95e+03
No. Iterations:                     9   Pseudo R-squ. (CS):             0.1256
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    -11.5405      0.435    -26.544      0.0

1.B


In [19]:
def boot_fn(D, idx):
    data = D.loc[idx]
    x = data[['income', 'balance']]
    y = data['default'] == "Yes"
    x_i = MS(x).fit_transform(x)
    model = sm.GLM(y, x_i, family=sm.families.Binomial()).fit()
    return model.params

1.C

In [20]:
B = 1000
first_=0
second_ = 0
data = load_data('Default')
for _ in range(B):
    idx = np.random.choice(data.shape[0], 1000, replace=True)
    value = boot_fn(data, idx)
    first_ += value
    second_ += value**2
SE = np.sqrt(second_ / B - (first_ / B)**2)
SE

intercept    1.590178
income       0.000016
balance      0.000825
dtype: float64

1.D

GLM guesses the error using fixed rules and math. Bootstrap checks by testing your real data many times, so it can spot hidden surprises that GLM might miss.

2.A & B

In [21]:
week_data = load_data('Weekly')
week_data

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up
...,...,...,...,...,...,...,...,...,...
1084,2010,-0.861,0.043,-2.173,3.599,0.015,3.205160,2.969,Up
1085,2010,2.969,-0.861,0.043,-2.173,3.599,4.242568,1.281,Up
1086,2010,1.281,2.969,-0.861,0.043,-2.173,4.835082,0.283,Up
1087,2010,0.283,1.281,2.969,-0.861,0.043,4.454044,1.034,Up


In [22]:
y = week_data['Direction'] == 'Up'
X = sm.add_constant(week_data[['Lag1', 'Lag2']])
model = sm.GLM(y, X, family=sm.families.Binomial()).fit()
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              Direction   No. Observations:                 1089
Model:                            GLM   Df Residuals:                     1086
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -744.11
Date:                Wed, 15 Oct 2025   Deviance:                       1488.2
Time:                        14:06:59   Pearson chi2:                 1.09e+03
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007303
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2212      0.061      3.599      0.0

In [23]:
X_train = X.iloc[1:]
y_train = y.iloc[1:]
model_first = sm.GLM(y_train, X_train, family=sm.families.Binomial()).fit()
print(model_first.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              Direction   No. Observations:                 1088
Model:                            GLM   Df Residuals:                     1085
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -743.26
Date:                Wed, 15 Oct 2025   Deviance:                       1486.5
Time:                        14:06:59   Pearson chi2:                 1.09e+03
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007373
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2232      0.061      3.630      0.0

In [24]:
prb = model.predict(X.iloc[[0]])[0]
pred_direction = 'Up' if prb > 0.5 else 'Down'
print(f"Predicted: {pred_direction}")

Predicted: Up


2.C
 

In [29]:
n = X.shape[0]
errors = np.zeros(n)
for i in range(n):
    X_train = X.iloc[np.arange(n) != i]
    y_train = y.iloc[np.arange(n) != i]
    model_loocv = sm.GLM(y_train, X_train, family=sm.families.Binomial()).fit()
    predict = model_loocv.predict(X.iloc[[i]])[0]
    pred = predict > 0.5
    errors[i] = int(pred != y.iloc[i])
loocv_error = errors.mean()


KeyError: 0

In [26]:
print(f"LOOCV estimate for the test error: {loocv_error:.4f}")

NameError: name 'loocv_error' is not defined